## Selection Methods

```json
{"id": 123, "custom": [{"key": "a", "value": "123"}, {"key": "b", "value": "yolo"}]}
```

vs.

```json
{"id": 123, "custom": {"key": "a", "value": "123"}}
{"id": 123, "custom": {"key": "b", "value": "yolo"}}
```

Database normal forms: https://www.guru99.com/database-normalization.html

In [ ]:
from typing import Dict, List
from jsonschema.exceptions import ValidationError
from jsonschema.validators import Draft3Validator
from jsonschema import validate

def translate(df: list, json_schema: dict) -> dict:
    v = Draft3Validator(json_schema)
    for record in df:
        for error in sorted(v.iter_errors(record), key=str):
            yield record, error

def parse_unexpected_keys(instance: dict, e: ValidationError) -> List[str]:
    '''
    "Additional properties are not allowed ('a' was unexpected)" -> ['a']
    "Additional properties are not allowed ('a', 'c' were unexpected)" -> ['a', 'c']
    '''
    return (
        list(map(
            # Strip the single quote from the outside of each key
            lambda x: x.strip("'"),
            # Strip the message of the beginning/end, 
            e.args[0]
            .lstrip('Additional properties are not allowed (')
            .rstrip(' was unexpected)')
            .rstrip(' were unexpected)')
            # split on comma e.g. "('a', 'c' was...)" -> ["'a'", "'b'"]
            .split(', ')
        ))
    )
            
for record, error in translate(records, json_schema):
    # print(error)
    ppd({
        'instance': record,
        'error': error.message,
        'keys': parse_unexpected_keys(record, error)
    })

In [ ]:
from dataclasses import dataclass, field

@dataclass
class UnexpectedKeys:
    schema: dict
    unique_keys: set = field(default_factory=set)
    
    def __post_init__(self):
        self.validator = Draft3Validator(self.schema)

    @property
    def keys(self):
        return list(sorted(self.unique_keys))

    @staticmethod
    def parse_unexpected_keys(e: ValidationError) -> List[str]:
        '''
        "Additional properties are not allowed ('a' was unexpected)" -> ['a']
        "Additional properties are not allowed ('a', 'c' were unexpected)" -> ['a', 'c']
        '''
        return (
            list(map(
                # Strip the single quote from the outside of each key
                lambda x: x.strip("'"),
                # Strip the message of the beginning/end, 
                e.args[0]
                .lstrip('Additional properties are not allowed (')
                .rstrip(' was unexpected)')
                .rstrip(' were unexpected)')
                # split on comma e.g. "('a', 'c' was...)" -> ["'a'", "'b'"]
                .split(', ')
            ))
        )

    
    def check(self, instance: dict):
        for error in sorted(self.validator.iter_errors(instance), key=str):
            for key in UnexpectedKeys.parse_unexpected_keys(error):
                self.unique_keys.add(key)
            yield error

In [ ]:
def split_dict(d: dict, keys: list):
    dd, extras = {}, {}
    for k in d.keys():
        if k in keys:
            extras[k] = d[k]
        else:
            dd[k] = d[k]
    return dd, extras

def translate(df: list, json_schema: dict) -> list:
    uk = UnexpectedKeys(json_schema)
    for record in records:
        for error in uk.check(record):
            d, extras = split_dict(record, UnexpectedKeys.parse_unexpected_keys(error))
            if extras:
                for k, v in extras.items():
                    yield {
                        **d, **{"custom": {"key": k, "value": v}}
                    }
            else:
                yield {**d, **{"custom": {}}}
            break
        else:
            yield {**record, **{"custom": {}}}

In [1]:
records = [
        {"id": 123, "a": "123", "b": "yolo"},
        {"id": 345, "a": "123", "z": "yolo"},
        {"id": 345, "a": "123", "b": "yolo2"},
]

records = [
    {"id": 123, "custom": [{"key": "a", "value": "123"}, {"key": "b", "value": "yolo"}]},
    {"id": 123, "custom": [{"key": "a", "value": "123"}, {"key": "z", "value": "yolo"}]},
    {"id": 123, "custom": [{"key": "a", "value": "123"}, {"key": "b", "value": "yolo2"}]},
]

In [2]:
import spark_utils, vis_utils
SPARK = spark_utils.spark_context()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/30 04:55:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/30 04:55:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = vis_utils.DFLoader.from_file(records)
df.show()

,custom,id
0,"[(a, 123), (b, yolo)]",123
1,"[(a, 123), (z, yolo)]",123
2,"[(a, 123), (b, yolo2)]",123


cells 6 / nulls 0
{
  "fields": [
    {
      "metadata": {},
      "name": "custom",
      "nullable": true,
      "type": {
        "containsNull": true,
        "elementType": {
          "fields": [
            {
              "metadata": {},
              "name": "key",
              "nullable": true,
              "type": "string"
            },
            {
              "metadata": {},
              "name": "value",
              "nullable": true,
              "type": "string"
            }
          ],
          "type": "struct"
        },
        "type": "array"
      }
    },
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
      "type": "long"
    }
  ],
  "type": "struct"
}
+--------------------+---+
|              custom| id|
+--------------------+---+
|[{a, 123}, {b, yo...|123|
|[{a, 123}, {z, yo...|123|
|[{a, 123}, {b, yo...|123|
+--------------------+---+



In [4]:
from pyspark.sql import functions as F

In [5]:
df.select(
    "id", F.explode("custom").alias("custom")
).filter(
    F.col("custom.key") == "b"
).select(
    "id", "custom.value"
).show()

+---+-----+
| id|value|
+---+-----+
|123| yolo|
|123|yolo2|
+---+-----+



In [6]:
records = [
    {"id": 123, "custom": {"key": "a", "value": "123"}},
    {"id": 123, "custom": {"key": "b", "value": "yolo"}},
    {"id": 321, "custom": {"key": "a", "value": "123"}},
    {"id": 321, "custom": {"key": "z", "value": "yolo"}},
    {"id": 555, "custom": {"key": "a", "value": "123"}},
    {"id": 555, "custom": {"key": "b", "value": "yolo2"}},
]
df = vis_utils.DFLoader.from_file(records)

,custom,id
0,"(a, 123)",123
1,"(b, yolo)",123
2,"(a, 123)",321
3,"(z, yolo)",321
4,"(a, 123)",555
5,"(b, yolo2)",555


cells 12 / nulls 0
{
  "fields": [
    {
      "metadata": {},
      "name": "custom",
      "nullable": true,
      "type": {
        "fields": [
          {
            "metadata": {},
            "name": "key",
            "nullable": true,
            "type": "string"
          },
          {
            "metadata": {},
            "name": "value",
            "nullable": true,
            "type": "string"
          }
        ],
        "type": "struct"
      }
    },
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
      "type": "long"
    }
  ],
  "type": "struct"
}


In [7]:
def select1(df):
    result = df.select(
        "id", F.explode("custom").alias("custom")
    ).filter(
        F.col("custom.key") == "b"
    ).select(
        "id", "custom.value"
    )
    result.count()
    return result

def select2(df):
    result = df.filter(
        F.col("custom.key") == "b"
    ).select(
        "id", "custom.value"
    )
    result.count()
    return result

In [8]:
from random import randint
import itertools

def random_records1(n, m=4):
    for i in range(n):
        extras = [{"key": "a", "value": randint(0, 122)}]
        for j in range(randint(0, m)):
            extras.append({"key": chr(randint(97, 122)), "value": randint(0, 122)})
        yield {"id": 123, "custom": extras}
        
def random_records2(n, m=4):
    for i in range(n):
        extras = [{"key": "a", "value": randint(0, 122)}]
        for j in range(randint(0, m)):
            extras.append({"key": chr(randint(97, 122)), "value": randint(0, 122)})
        for extra in extras:
            yield {"id": 123, "custom": extra}

In [9]:
import time

In [12]:
df1 = vis_utils.DFLoader.from_file(random_records1(10_000, 200))

start = time.time()
select1(df1)
print(time.time() - start)
df1 = None

,custom,id
0,"[(a, 40), (n, 74), (y, 20), (s, 80), (v, 67), ...",123
1,"[(a, 46), (v, 46), (l, 103), (x, 44), (p, 62),...",123
2,"[(a, 13), (g, 24), (d, 93), (v, 120), (t, 100)...",123
3,"[(a, 78), (z, 97), (i, 66), (o, 20), (v, 114),...",123
4,"[(a, 60), (p, 81), (b, 108), (a, 102), (i, 85)...",123
...,...,...
9995,"[(a, 41), (g, 100), (t, 4), (x, 56), (y, 115),...",123
9996,"[(a, 43), (c, 91), (r, 0), (x, 45), (k, 4), (o...",123
9997,"[(a, 56), (l, 60), (n, 78), (f, 106), (d, 103)...",123
9998,"[(a, 93), (p, 18), (w, 85), (s, 90), (o, 74), ...",123


cells 20000 / nulls 0
{
  "fields": [
    {
      "metadata": {},
      "name": "custom",
      "nullable": true,
      "type": {
        "containsNull": true,
        "elementType": {
          "fields": [
            {
              "metadata": {},
              "name": "key",
              "nullable": true,
              "type": "string"
            },
            {
              "metadata": {},
              "name": "value",
              "nullable": true,
              "type": "long"
            }
          ],
          "type": "struct"
        },
        "type": "array"
      }
    },
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
      "type": "long"
    }
  ],
  "type": "struct"
}
0.6988532543182373


In [14]:
df2 = vis_utils.DFLoader.from_file(random_records2(10_000, 200))

start = time.time()
select2(df2)
print(time.time() - start)
df1 = None

,custom,id
0,"(a, 93)",123
1,"(a, 81)",123
2,"(z, 86)",123
3,"(u, 95)",123
4,"(r, 101)",123
...,...,...
1013619,"(r, 77)",123
1013620,"(p, 49)",123
1013621,"(d, 43)",123
1013622,"(k, 90)",123


cells 2027248 / nulls 0
{
  "fields": [
    {
      "metadata": {},
      "name": "custom",
      "nullable": true,
      "type": {
        "fields": [
          {
            "metadata": {},
            "name": "key",
            "nullable": true,
            "type": "string"
          },
          {
            "metadata": {},
            "name": "value",
            "nullable": true,
            "type": "long"
          }
        ],
        "type": "struct"
      }
    },
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
      "type": "long"
    }
  ],
  "type": "struct"
}


1.3485691547393799
